# PITCH TRACKING

In [1]:
import ode
import numpy as np
import matplotlib.pyplot as plt
from vpython import *
import csv

<IPython.core.display.Javascript object>

In [ ]:
def readData(filename = "savant_data.csv"):
    #default dataset is savant.csv, user can pass in whatever they want
    
    #creates the necessary lists
    date = [] 
    release_speed = [] #in mph
    release_pos = [] #vectors of x0, y0, v0
    player_name = []
    event = []
    result = [] #result of the pitch
    balls = [] #current # of balls
    strikes = [] #current # of strikes
    pitch_name = [] #pitch type
    release_vel = [] #vectors of vx0, vy0, and vz0
    spin_axis = [] #spin axis in 2D x-z plane, such that 180 is pure backspin and 0 is pure topspin
    release_spin = [] #spin rate of pitch (rpm)
    
    with open(filename) as file:
        rawdata = csv.reader(file, delimiter = ',')
        next(file)
        for line in rawdata:
            date.append(line[1])
            release_speed.append(line[2])
            release_pos_x = line[3]
            release_pos_y = line[68]
            release_pos_z = line[4]
            release_pos.append(vector(float(release_pos_x),float(release_pos_y),float(release_pos_z)))
            player_name.append(line[5])
            event.append(line[8])
            result.append(line[9])
            balls.append(line[24])
            strikes.append(line[25])
            vx0 = line[44]
            vy0 = line[45]
            vz0 = line[46]
            release_vel.append(vector(float(vx0),float(vy0),float(vz0)))
            pitch_name.append(line[78])
            spin_axis.append(line[89])
            release_spin.append(line[56])
            
    return date, release_speed, release_pos, player_name, event, result, balls, strikes, release_vel, pitch_name, spin_axis, release_spin

In [ ]:
#Data reading and handling

date = [] 
release_speed = [] #in mph
release_pos = [] #vectors of x0, y0, v0
player_name = []
event = []
result = [] #result of the pitch
balls = [] 
strikes = [] 
pitch_name = [] #pitch type
release_vel = [] #vectors of vx0, vy0, and vz0
spin_axis = [] #spin axis in 2D x-z plane, such that 180 is pure backspin and 0 is pure topspin
release_spin = [] #spin rate of pitch (rpm)

date, release_speed, release_pos, player_name, event, result, balls, strikes, release_vel, pitch_name, spin_axis, release_spin = readData()

date_list = []
for x in date:
    if x not in date_list:
        date_list.append(x)

print("Pitcher:", player_name[0], "\nDates: ", date_list)
y = len(date)-1

for x in range(0,len(date)):
    result[y-x] = result[y-x].replace("_", " ")
    event[y-x] = event[y-x].replace("_", " ")
    if (all(a.isalpha() or a.isspace() for a in event[y-x]) and event[y-x]):
        event[y-x] = '(' + event[y-x] + ')'
    print("Pitch:", '{0: <18}'.format(pitch_name[y-x]), '{0: <3}'.format(release_speed[y-x]), "MPH   Count:", balls[y-x], "-", strikes[y-x], " Result:", result[y-x], event[y-x])

In [2]:
def getCd(v):
    # calculate value of drag coefficient for a particular speed and case
    case = 2
    if case == 0:
        Cd = 0
    elif case == 1:
        Cd = 0.5
    elif case == 2:
        a = 0.36
        b = 0.14
        c = 0.27
        vc = 34
        chi = (v - vc)/4
        if chi < 0:
            Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2)
        else:
            Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2/4)
    else:
        Cd = 0.5
    
    return Cd

def arraymag(v):
    #calculate magnitude of an array
    return np.sqrt(np.dot(v,v))

def arrayhat(v):
    #calculate unit vector of an array
    return v/mag(v)

def arraycross(v1,v2):
    #calculate the cross product of two vectors
    return np.cross(v1,v2)

def model_magnus(d, tn):
    #return array of derivatives
        
    #data
    x = d[0]
    y = d[1]
    z = d[2]  
    vx = d[3]
    vy = d[4]
    vz = d[5]
    
    #derivatives
    rate = np.zeros(6) #derivatives
    rate[0] = vx
    rate[1] = vy
    rate[2] = vz
    
    #speed
    v = np.array([vx,vy,vz])
    vmag = arraymag(v)
    
    #calculate force and dv/dt
    Cd=getCd(vmag)
    b2 = 1/2*Cd*rho*A
    
    ωmag = float(srate) * 2*np.pi / 60 #rad/s
    ω = np.array([sin(ωmag), 0, cos(ωmag)])
    S = r*ωmag/vmag
    CL = 0.62*S**0.7
    alpha = 1/2*CL*rho*A*r/S
    
    FM = alpha*arraycross(ω,v) #magnus force
    
    rate[3] = (-b2*vmag*vx)/m + FM[0]/m
    rate[4] = (-b2*vmag*vy)/m + FM[1]/m
    rate[5] = (-b2*vmag*vz - (m*g))/m  + FM[2]/m
    
    return rate


def runSim(posvec,vel,spin_rate,spin_axis):
    global b2, alpha #need to change the value of b2 and alpha
    #time
    t = 0
    h= 0.01
    Nsteps = int(10/h)
    
    results = []
    
    #convert f/s to m/s
    #vel.x = vel.x*0.3048
    print(vel.x)
    #vel.y = vel.y*0.3048
    print(vel.y)
    #vel.z = vel.z*0.3048
    print(vel.z)
    #convert deg to rad
    #print(spin_axis)
    theta = float(spin_axis)*np.pi/180 
    #print(theta)
    
    #print(spin_rate)
    global srate
    srate = spin_rate
    #print(srate)
    ωmag = float(spin_rate) * 2*np.pi / 60 #rad/s
    ω = np.array([sin(ωmag), 0, cos(ωmag)])
    #print(ωmag)
    
   


    data = np.array([posvec.x,posvec.y,posvec.z,vel.x,vel.y,vel.z])
    
    #store trajectory for plotting or animation

    for n in range(0,Nsteps-1):

        #update data
        data = ode.RK4(model_magnus, data, t, h)
        
        #update t
        t=t+h

        #store data for plotting
        results.append(vec(data[0],data[1],data[2]))
    
    return results
    
    
    

In [ ]:
scene1 = canvas(title="Pitch Tracking", autoscale=False)
scene1.background = color.gray(0.6)
ground = box(pos=vec(-1,-1,-10.1), size=vec(200,200,20), color=color.green)
home = box(pos=vec(0,0,0), size=vec(0.43177968,0.43177968,0.0762), color=color.white)
pitchbox = box(pos=vec(0.3048,18.4404,0.24384), size=vec(0.6096,0.1524,0.0762))
scene1.center = vec(0,9.2202,0.1524)
scene1.camera.axis = vec(0.0190896, 18.0867, -1.64663)
scene1.camera.pos = vec(0.688913, -5.3547, 2.64525)
#scene1.camera.rotate(angle=1/2)
scene1.pause()

pitches = []

Nsteps = int(10/0.01)
posarray=[vec(0,0,0)]*len(date)
#print(posarray)

#parameters of the system

g = 9.8 #N/kg
rho = 1.2 #kg/m^3
mu = 1.8e-5 #kg/m/s
r = 74e-3/2 #74 mm diameter, 9.25" in circumference
A = np.pi*r**2 #cross-sectional area
m = 0.145 #kg
Cd = 0.35 #actually depends on speed
b2 = 1/2*Cd*rho*A #will change as Cd changes
S = 0.01 #will change as omega and v change
CL = 0 #will change with S
alpha = 1/2*CL*rho*A*r/S
y=len(date)-1

#print(len(date))
for x in range(0,4):
    #print(x)
    #print(posarray[0][0])
    posarray[y-x] = vec(release_pos[y-x].x*0.3048,release_pos[y-x].y*0.3048,release_pos[y-x].z*0.3048)
    positions = []
    #get simulation data
    data = runSim(posarray[y-x],release_vel[y-x],release_spin[y-x],spin_axis[y-x])
    for b in range(1,len(data)):
        positions.append(data[b-1])
    
    #show in visualization
    ball = sphere(pos = posarray[y-x], radius = r, color = color.white, make_trail=True)
    pitches.append(ball)
    
    z=0
    while(pitches[x].pos.y >= 0 and pitches[x].pos.z >= 0 and z<len(positions)):
        rate(100)
        z=z+1
        pitches[x].pos = positions[z]
    
    #print(date[y-x])
    print("Pitch:", '{0: <18}'.format(pitch_name[y-x]), '{0: <3}'.format(release_speed[y-x]), "MPH   Count:", balls[y-x], "-", strikes[y-x], " Result:", result[y-x], event[y-x])
    scene1.pause()
    pitches[x].clear_trail()
    pitches[x].visible = False
        
for c in range(len(pitches)):
    #print(pitches[c].pos)
    pitches[c].visible = True
    

In [6]:
def readData(filename = "kershaw_data.csv"):
    #default dataset is savant.csv, user can pass in whatever they want
    
    #creates the necessary lists
    date = [] 
    release_speed = [] #in mph
    release_pos = [] #vectors of x0, y0, v0
    player_name = []
    event = []
    result = [] #result of the pitch
    balls = [] #current # of balls
    strikes = [] #current # of strikes
    pitch_name = [] #pitch type
    release_vel = [] #vectors of vx0, vy0, and vz0
    spin_axis = [] #spin axis in 2D x-z plane, such that 180 is pure backspin and 0 is pure topspin
    release_spin = [] #spin rate of pitch (rpm)
    
    with open(filename) as file:
        rawdata = csv.reader(file, delimiter = ',')
        next(file)
        for line in rawdata:
            date.append(line[1])
            release_speed.append(line[2])
            release_pos_x = line[3]
            release_pos_y = line[68]
            release_pos_z = line[4]
            release_pos.append(vector(float(release_pos_x),float(release_pos_y),float(release_pos_z)))
            player_name.append(line[5])
            event.append(line[8])
            result.append(line[9])
            balls.append(line[24])
            strikes.append(line[25])
            vx0 = line[44]
            vy0 = line[45]
            vz0 = line[46]
            release_vel.append(vector(float(vx0),float(vy0),float(vz0)))
            pitch_name.append(line[78])
            spin_axis.append(line[89])
            release_spin.append(line[56])
            
    return date, release_speed, release_pos, player_name, event, result, balls, strikes, release_vel, pitch_name, spin_axis, release_spin

date = [] 
release_speed = [] #in mph
release_pos = [] #vectors of x0, y0, v0
player_name = []
event = []
result = [] #result of the pitch
balls = [] 
strikes = [] 
pitch_name = [] #pitch type
release_vel = [] #vectors of vx0, vy0, and vz0
spin_axis = [] #spin axis in 2D x-z plane, such that 180 is pure backspin and 0 is pure topspin
release_spin = [] #spin rate of pitch (rpm)

date, release_speed, release_pos, player_name, event, result, balls, strikes, release_vel, pitch_name, spin_axis, release_spin = readData()

date_list = []
for x in date:
    if x not in date_list:
        date_list.append(x)

print("Pitcher:", player_name[0], "\nDates: ", date_list)
y = len(date)-1

for x in range(0,len(date)):
    result[y-x] = result[y-x].replace("_", " ")
    event[y-x] = event[y-x].replace("_", " ")
    if (all(a.isalpha() or a.isspace() for a in event[y-x]) and event[y-x]):
        event[y-x] = '(' + event[y-x] + ')'
        
def getCd(v):
    # calculate value of drag coefficient for a particular speed and case
    case = 2
    if case == 0:
        Cd = 0
    elif case == 1:
        Cd = 0.5
    elif case == 2:
        a = 0.36
        b = 0.14
        c = 0.27
        vc = 34
        chi = (v - vc)/4
        if chi < 0:
            Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2)
        else:
            Cd = a + b/(1+np.exp(chi)) - c*np.exp(-chi**2/4)
    else:
        Cd = 0.5
    
    return Cd

def arraymag(v):
    #calculate magnitude of an array
    return np.sqrt(np.dot(v,v))

def arrayhat(v):
    #calculate unit vector of an array
    return v/mag(v)

def arraycross(v1,v2):
    #calculate the cross product of two vectors
    return np.cross(v1,v2)

def model_magnus(d, tn):
    #return array of derivatives
        
    #data
    x = d[0]
    y = d[1]
    z = d[2]  
    vx = d[3]
    vy = d[4]
    vz = d[5]
    
    #derivatives
    rate = np.zeros(6) #derivatives
    rate[0] = vx
    rate[1] = vy
    rate[2] = vz
    
    #speed
    v = np.array([vx,vy,vz])
    vmag = arraymag(v)
    
    #calculate force and dv/dt
    Cd=getCd(vmag)
    b2 = 1/2*Cd*rho*A
    
    ωmag = float(srate) * 2*np.pi / 60 #rad/s
    ω = np.array([sin(ωmag), 0, cos(ωmag)])
    S = r*ωmag/vmag
    CL = 0.62*S**0.7
    alpha = 1/2*CL*rho*A*r/S
    
    FM = alpha*arraycross(ω,v) #magnus force
    
    rate[3] = (-b2*vmag*vx)/m + FM[0]/m
    rate[4] = (-b2*vmag*vy)/m + FM[1]/m
    rate[5] = (-b2*vmag*vz - (m*g))/m  + FM[2]/m
    
    return rate


def runSim(posvec,vel,spin_rate,spin_axis):
    global b2, alpha #need to change the value of b2 and alpha
    #time
    t = 0
    h= 0.0005
    Nsteps = int(10/h)
    
    results = []
    
    #convert f/s to m/s
    vel.x = vel.x*0.3048
    #print(vel.x)
    vel.y = vel.y*0.3048
    #print(vel.y)
    vel.z = vel.z*0.3048
    #print(vel.z)
    #convert deg to rad
    #print(spin_axis)
    theta = float(spin_axis)*np.pi/180 
    #print(theta)
    
    #print(spin_rate)
    global srate
    srate = spin_rate
    #print(srate)
    ωmag = float(spin_rate) * 2*np.pi / 60 #rad/s
    ω = np.array([sin(ωmag),0, cos(ωmag)])
    #print(ωmag)
    
   


    data = np.array([posvec.x,posvec.y,posvec.z,vel.x,vel.y,vel.z])
    
    #store trajectory for plotting or animation

    for n in range(0,Nsteps-1):

        #update data
        data = ode.RK4(model_magnus, data, t, h)
        
        #update t
        t=t+h

        #store data for plotting
        results.append(vec(data[0],data[1],data[2]))
    
    return results
    
    
scene1 = canvas(title="Pitch Tracking", autoscale=False)
scene1.background = color.gray(0.6)
ground = box(pos=vec(-1,-1,-10.3), size=vec(200,200,20), color=color.green)
home = box(pos=vec(0,0,-0.2), size=vec(0.43177968,0.43177968,0.0381), color=color.white)
pitchbox = box(pos=vec(0.3048,18.4404,0.24384), size=vec(0.6096,0.1524,0.0762))
scene1.center = vec(0,9.2202,0.1524)
scene1.camera.axis = vec(0.0190896, 18.0867, -1.64663)
scene1.camera.pos = vec(0.688913, -5.3547, 2.64525)
#scene1.camera.rotate(angle=1/2)
scene1.pause()

pitches = []

Nsteps = int(10/0.01)
posarray=[vec(0,0,0)]*len(date)
#print(posarray)

#parameters of the system

g = 9.8 #N/kg
rho = 1.2 #kg/m^3
mu = 1.8e-5 #kg/m/s
r = 74e-3/2 #74 mm diameter, 9.25" in circumference
A = np.pi*r**2 #cross-sectional area
m = 0.145 #kg
Cd = 0.35 #actually depends on speed
b2 = 1/2*Cd*rho*A #will change as Cd changes
S = 0.01 #will change as omega and v change
CL = 0 #will change with S
alpha = 1/2*CL*rho*A*r/S
y=len(date)-1

#print(len(date))
for x in range(0,5):
    #print(x)
    #print(posarray[0][0])
    posarray[y-x] = vec(release_pos[y-x].x*0.3048,release_pos[y-x].y*0.3048,release_pos[y-x].z*0.3048)
    positions = []
    #get simulation data
    data = runSim(posarray[y-x],release_vel[y-x],release_spin[y-x],spin_axis[y-x])
    for b in range(1,len(data)):
        positions.append(data[b-1])
    
    #show in visualization
    ball = sphere(pos = posarray[y-x], radius = r, color = color.white, make_trail=True)
    pitches.append(ball)
    
    z=0
    while(pitches[x].pos.y >= 0.43177968 and pitches[x].pos.z >= 0 and z<len(positions)):
        rate(300)
        z=z+1
        pitches[x].pos = positions[z]
    
    #print(date[y-x])
    print("Pitch:", '{0: <18}'.format(pitch_name[y-x]), '{0: <3}'.format(release_speed[y-x]), "MPH   Count:", balls[y-x], "-", strikes[y-x], " Result:", result[y-x], event[y-x])
    #scene1.pause()
    pitches[x].clear_trail()
    pitches[x].visible = False
        
for c in range(len(pitches)):
    #print(pitches[c].pos)
    pitches[c].visible = True
    

Pitcher: Kershaw, Clayton 
Dates:  ['2021-04-28', '2021-04-23', '2021-04-17', '2021-04-11', '2021-04-06', '2021-04-01']


<IPython.core.display.Javascript object>

Pitch: 4-Seam Fastball    90.6 MPH   Count: 0 - 0  Result: ball 
Pitch: 4-Seam Fastball    90.7 MPH   Count: 1 - 0  Result: ball 
Pitch: 4-Seam Fastball    91.2 MPH   Count: 2 - 0  Result: hit into play (field out)
Pitch: 4-Seam Fastball    92.0 MPH   Count: 0 - 0  Result: hit into play (field out)
Pitch: 4-Seam Fastball    90.7 MPH   Count: 0 - 0  Result: ball 
